In [36]:
from skopt import gp_minimize
from skopt.learning import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Kernel, _check_length_scale
from skopt.acquisition import gaussian_ei
import numpy as np
import networkx as nx
from typing import List, Tuple, Any

In [37]:
# Define graph search space
# graph = ...

In [38]:
# Define objective function with validation metric
def objective_function(params: Any) -> float:
    # Train the neural network with the given parameters
    validation_metric = train_neural_network(params)
    return -validation_metric  # Minimize the negative of the validation metric

# Compute the Laplacian matrix of the graph
def get_laplacian_matrix(graph: nx.Graph) -> np.ndarray:
    # Convert the graph to an adjacency matrix
    adjacency_matrix = nx.adjacency_matrix(graph).toarray()
    
    # Compute the degree matrix
    degree_matrix = np.diag(np.sum(adjacency_matrix, axis=1))
    
    # Compute the Laplacian matrix: L = D - A
    laplacian = degree_matrix - adjacency_matrix
    
    return laplacian

# Perform eigendecomposition of the Laplacian matrix
def get_eigendecomposition(laplacian: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    # Compute eigendecomposition: L = U * Lambda * U^T
    eigenvalues, eigenvectors = np.linalg.eigh(laplacian)
    return eigenvectors, np.diag(eigenvalues)

# Define the covariance matrix of the GP
def compute_covariance_matrix(eigenvectors: np.ndarray, eigenvalues: np.ndarray, beta: float) -> np.ndarray:
    # Define the covariance matrix: U * e^(-beta * Lambda) * U^T
    covariance_matrix = np.dot(eigenvectors, np.dot(np.exp(-beta * eigenvalues), eigenvectors.T))
    return covariance_matrix

# Create custom kernel for the GP
# https://stackoverflow.com/questions/49188159/how-to-create-a-custom-kernel-for-a-gaussian-process-regressor-in-scikit-learn
# https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/gaussian_process/kernels.py#L165
class LaplacianGPKernel(Kernel):
    def __init__(self, laplacian_matrix: np.ndarray, beta: float):
        self.laplacian_matrix = laplacian_matrix
        self.beta = beta

    def __call__(self, X: np.ndarray, Y: np.ndarray = None) -> np.ndarray:
        if Y is None:
            Y = X
        # Compute the Laplacian-based covariance matrix
        v, e = get_eigendecomposition(X)
        K = compute_covariance_matrix(v, e, self.beta)
        return K

    def diag(self, X: np.ndarray) -> np.ndarray:
        return np.ones(X.shape[0])

    def is_stationary(self) -> bool:
        return True

In [ ]:
# Example of Usage

# Generate a random graph
graph = nx.erdos_renyi_graph(10, 0.3)

# Compute the Laplacian matrix
laplacian = get_laplacian_matrix(graph)

# Perform eigendecomposition
eigenvectors, eigenvalues = get_eigendecomposition(laplacian)

# Define the covariance matrix of the GP
kernel = LaplacianGPKernel(laplacian)

# Initialize the custom GP with the defined covariance matrix
gp = GaussianProcessRegressor(kernel=LaplacianGPKernel)

# Initialize a list to store evaluated nodes and corresponding validation metrics
evaluated_nodes: List[Any] = []
evaluated_metrics: List[float] = []

# Run bayesian optimization
for _ in range(10):
    # Perform Bayesian optimization step with Expected Improvement
    result = gp_minimize(lambda x: -objective_function(x), graph.nodes(),
                         base_estimator=gp, acq_func="EI", n_calls=1)

    # Get the next node to evaluate
    next_node = result.x

    # Evaluate the objective function on the selected node
    validation_metric = objective_function(next_node)

    # Update the surrogate model with the new data
    evaluated_nodes.append(next_node)
    evaluated_metrics.append(validation_metric)
    gp.fit(np.array(evaluated_nodes).reshape(-1, 1), evaluated_metrics)

# Get the optimal node and validation metric found
optimal_node = evaluated_nodes[np.argmin(evaluated_metrics)]
optimal_metric = np.min(evaluated_metrics)

print("Optimal node:", optimal_node)
print("Optimal validation metric:", optimal_metric)